In [1]:
import pandas as pd
import numpy as np
import glob

# GSDC Raw Data Overview

In [2]:
glob.glob("./data/raw_data/*")

['./data/raw_data/all_compounds_screened.csv',
 './data/raw_data/GDSC2_drug_dose_cellines_IC50s.xlsx',
 './data/raw_data/cellines_rnaseq_all_20220624',
 './data/raw_data/celline_SNP6_cnv_gistics_20191101',
 './data/raw_data/all_cellines_screened.xlsx',
 './data/raw_data/cellines_rnaseq_all_20220624.zip',
 './data/raw_data/mutations_all_20230202.csv',
 './data/raw_data/mutations_summary_20230202.csv',
 './data/raw_data/cellines_mutations_summary_20230202.zip',
 './data/raw_data/celline_SNP6_cnv_gistics_20191101.zip',
 './data/raw_data/drug_info.csv',
 './data/raw_data/cellines_mutations_all_20230202.zip']

In [3]:
all_compounds_screened = pd.read_csv("./data/raw_data/all_compounds_screened.csv")
all_cellines_screened = pd.read_excel("./data/raw_data/all_cellines_screened.xlsx", sheet_name=0)
all_experiment = pd.read_excel("./data/raw_data/GDSC2_drug_dose_cellines_IC50s.xlsx")

/Users/sunzehui/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
# 选取有测序结果和甲基化数据等数据的细胞系
filtered_celline = all_cellines_screened.loc[
    (all_cellines_screened['Whole Exome Sequencing (WES)'] == "Y") &
    (all_cellines_screened['Methylation'] == "Y") &
    (all_cellines_screened['Gene Expression'] == "Y") &
    (all_cellines_screened['Copy Number Alterations (CNA)'] == "Y") &
    (all_cellines_screened['Drug\nResponse'] == "Y")
]
filtered_celline.head()

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [5]:
filtered_celline.head()

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [6]:
a = set(all_experiment['CELL_LINE_NAME'])
b = set(filtered_celline['Sample Name'])
print(f"experiment sheet includes {a.__len__()} unqiue cellines")
print(f"all_celline sheet includes {b.__len__()} unique cellines")
celline_barcode = list(a.intersection(b))
print(f"two sheets have {celline_barcode.__len__()} overlapping cellines")

experiment sheet includes 969 unqiue cellines
all_celline sheet includes 928 unique cellines
two sheets have 870 overlapping cellines


In [7]:
filtered_experiment = all_experiment[all_experiment['CELL_LINE_NAME'].isin(celline_barcode)]
filtered_experiment.head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038


# Single Celline Multi-omics data Overview

## Copy Number Variation

In [8]:
all_cna = pd.read_csv("./data/raw_data/celline_SNP6_cnv_gistics_20191101/cnv_abs_copy_number_picnic_20191101.csv", low_memory=False)
all_fpkm = pd.read_csv("./data/raw_data/cellines_rnaseq_all_20220624/rnaseq_fpkm_20220624.csv", low_memory=False)

In [9]:
# Doing some preprocessing
dat = all_cna.iloc[2:24505]
titile = all_cna.iloc[[0]]
df = pd.concat([titile, dat])
df = df.T.iloc[1:]
new_header = df.iloc[[0]].values
new_header = new_header.tolist()
new_header = new_header[0]
new_header[0] = "celline_barcode"
df.columns = new_header
df = df.iloc[1:].set_index("celline_barcode")

In [10]:
df = df.fillna(0.0)
cna_df = df

## Gene Expression (FPKM-based)

In [11]:
dat = all_fpkm.iloc[4:]
titile = all_fpkm.iloc[[0]]
df = pd.concat([titile, dat])
df = df.T.iloc[1:]
df
new_header = df.iloc[[0]].values
new_header = new_header.tolist()
new_header = new_header[0]
new_header[0] = "celline_barcode"
df.columns = new_header
df = df.iloc[1:].set_index("celline_barcode")

In [12]:
df = df.fillna(0.0)
fpkm_df = df

## Datasets Alignment

In [13]:
s1 = set(fpkm_df.index).intersection(celline_barcode)
s2 = set(cna_df.index).intersection(celline_barcode)
celline_barcode = list(s1.intersection(s2))
print(f"Two datasets have {len(celline_barcode)} common cellines")

Two datasets have 866 common cellines


In [14]:
common_genes = list(set(cna_df.columns).intersection(fpkm_df.columns))
print(f"Two datasets have {len(common_genes)} genes(features)")

Two datasets have 23914 genes(features)


In [15]:
fpkm_df.loc[celline_barcode].to_csv("./data/processed_data/fpkm.csv", sep='\t')
cna_df.loc[celline_barcode].to_csv("./data/processed_data/cna.csv", sep='\t')
fpkm_df = fpkm_df.loc[celline_barcode]
cna_df = cna_df.loc[celline_barcode]

In [16]:
filtered_experiment[filtered_experiment['CELL_LINE_NAME'].isin(celline_barcode)].head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038


## GDSC Dataset

Filter dataset

In [17]:
# drug_info: https://www.cancerrxgene.org/downloads/drug_data
drug_df = pd.read_csv('./data/raw_data/drug_info.csv', sep=',')
drug_df.head()

,drug_id,drug_name,synonyms,pathway_name,targets,pubchem
0,1834,743380,-,Unclassified,-,-
1,1854,MN-64,-,WNT signaling,"TNKS1, TNKS2",2802462
2,1909,Venetoclax,"ABT-199, Veneclexta, GDC-0199",Apoptosis regulation,BCL2,49846579
3,1913,AGI-5198,IDH-C35,Metabolism,IDH1 (R132H),56645356
4,1915,AZD3759,-,EGFR signaling,EGFR,78209992


In [18]:
# pubchem_id exclude non-numeric rows
import re
nonnumber = re.compile(r'\D+')
pubchem_id = list(set(drug_df['pubchem']))
pubchem_id = [i.split(',')[0] if "," in i else i for i in pubchem_id]
pubchem_id = [i for i in pubchem_id if re.findall(pattern=nonnumber, string=i).__len__()==0]
drug_df = drug_df[drug_df['pubchem'].isin(pubchem_id)]

In [19]:
all_drug_in_experiment = list(set(all_experiment['DRUG_NAME']))
d = drug_df[drug_df['drug_name'].isin(all_drug_in_experiment)][['drug_name', 'pubchem']]

In [20]:
lookup_table = dict(zip(d['drug_name'], d['pubchem']))
all_experiment = all_experiment[all_experiment['DRUG_NAME'].isin(d['drug_name'])]
all_experiment['PUBCHEM_ID'] = [lookup_table[i] for i in all_experiment['DRUG_NAME']]


In [21]:
all_experiment = all_experiment[all_experiment['CELL_LINE_NAME'].isin(celline_barcode)]
all_experiment.reset_index(inplace = True)
all_experiment.drop(columns = "index", inplace = True)
all_experiment

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,PUBCHEM_ID
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-1.462148,0.930105,0.088999,0.432482,24360
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-4.869447,0.614932,0.111423,-1.420322,24360
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.360684,0.790953,0.142754,-0.599894,24360
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-5.045014,0.592624,0.135642,-1.515791,24360
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.741620,0.733992,0.128066,-0.807038,24360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145002,GDSC2,401,19186874,1659818,MM1S,SIDM01265,MM,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,4.290123,0.991533,0.020244,-0.962882,70983932
145003,GDSC2,401,19187483,1659928,SNU-175,SIDM00216,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,5.423870,0.985066,0.032710,0.001082,70983932
145004,GDSC2,401,19187936,1660034,SNU-407,SIDM00214,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,5.042005,0.970009,0.026729,-0.323597,70983932
145005,GDSC2,401,19188194,1660035,SNU-61,SIDM00194,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,6.130028,0.986298,0.027488,0.601490,70983932


In [22]:
all_experiment.to_csv("./data/processed_data/expriment.csv", sep="\t", index = None)

## Response and Pubchem ID troubleshooting

In [23]:
import pubchempy as pcp
df = pcp.get_properties(properties=['canonical_smiles'], identifier=pubchem_id,
                        namespace='cid', )
df = pd.DataFrame(df)
df[['CID']]=df[['CID']].astype(str)
df.to_csv("./data/processed_data/pubchem_id-SMILES.csv", sep='\t')
lookup_table_cid_smiles = dict(zip(df['CID'], df['CanonicalSMILES']))

In [24]:
all_experiment = all_experiment[all_experiment['PUBCHEM_ID'].isin(pubchem_id)]
all_experiment['SMILES']=[lookup_table_cid_smiles[i] for i in all_experiment['PUBCHEM_ID']]

In [25]:
all_experiment.head(5)

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,...,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,PUBCHEM_ID,SMILES
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...


In [26]:
sample_barcode = [f"{i[0]}_{i[1]}" for i in zip(all_experiment['CELL_LINE_NAME'], all_experiment['PUBCHEM_ID'])]

In [27]:
response = pd.DataFrame()
response['sample_barcode'] = sample_barcode
response['LN_IC50'] = all_experiment['LN_IC50']
response.reset_index(drop=True, inplace=True)

In [28]:
response.to_csv('./data/processed_data/response.csv', sep='\t', index=None)
all_experiment.to_csv('./data/processed_data/expriment.csv', sep='\t', index=None)

## Drug Feature Extraction using beta-VAE

In [29]:
import pandas as pd
from os.path import realpath
import pandas as pd
from keras.models import load_model
import ast
import pandas as pd
import numpy as np
from rdkit import Chem, RDLogger
from rdkit.Chem import BondType

RDLogger.DisableLog("rdApp.*")

SMILE_CHARSET = '["C", "B", "F", "I", "H", "O", "N", "S", "P", "Cl", "Br"]'

bond_mapping = {"SINGLE": 0, "DOUBLE": 1, "TRIPLE": 2, "AROMATIC": 3}
bond_mapping.update(
    {0: BondType.SINGLE, 1: BondType.DOUBLE, 2: BondType.TRIPLE, 3: BondType.AROMATIC}
)
SMILE_CHARSET = ast.literal_eval(SMILE_CHARSET)

SMILE_to_index = dict((c, i) for i, c in enumerate(SMILE_CHARSET))
index_to_SMILE = dict((i, c) for i, c in enumerate(SMILE_CHARSET))
atom_mapping = dict(SMILE_to_index)
atom_mapping.update(index_to_SMILE)

BATCH_SIZE = 100
EPOCHS = 10

VAE_LR = 5e-4
NUM_ATOMS = 120  # Maximum number of atoms

ATOM_DIM = len(SMILE_CHARSET)  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 435  # Size of the latent space


def smiles_to_graph(smiles):
    # Converts SMILES to molecule object
    molecule = Chem.MolFromSmiles(smiles)

    # Initialize adjacency and feature tensor
    adjacency = np.zeros((BOND_DIM, NUM_ATOMS, NUM_ATOMS), "float32")
    features = np.zeros((NUM_ATOMS, ATOM_DIM), "float32")

    # loop over each atom in molecule
    # Ignore Pt Atom
    for atom in molecule.GetAtoms():
        if(atom.GetSymbol() == "Pt"):
            continue
        i = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        features[i] = np.eye(ATOM_DIM)[atom_type]
        # loop over one-hop neighbors
        for neighbor in atom.GetNeighbors():
            j = neighbor.GetIdx()
            bond = molecule.GetBondBetweenAtoms(i, j)
            bond_type_idx = bond_mapping[bond.GetBondType().name]
            adjacency[bond_type_idx, [i, j], [j, i]] = 1

    # Where no bond, add 1 to last channel (indicating "non-bond")
    # Notice: channels-first
    adjacency[-1, np.sum(adjacency, axis=0) == 0] = 1

    # Where no atom, add 1 to last column (indicating "non-atom")
    features[np.where(np.sum(features, axis=1) == 0)[0], -1] = 1

    return adjacency, features


def graph_to_molecule(graph):
    # Unpack graph
    adjacency, features = graph

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0)
    )[0]
    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    # Add bonds between atoms in molecule; based on the upper triangles
    # of the [symmetric] adjacency tensor
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    # Let's be strict. If sanitization fails, return None
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule


# 注：min-Max归一化需要在分割完训练集和测试集和Validation set之后再进行

# Cached_data
cached_data = {}

# Path Define
cna_path = realpath("data/processed_data/cna.csv")
experiment_path = realpath("data/processed_data/experiment.csv")
fpkm_path = realpath("data/processed_data/fpkm.csv")
SMILES_path = realpath("data/processed_data/pubchem_id-SMILES.csv")

drug_AdjacencyTensor = []
drug_FeatureTensor = []

df = pd.read_csv(SMILES_path, sep='\t')
for i in df["CanonicalSMILES"]:
    _ad, _fe = smiles_to_graph(i)
    drug_AdjacencyTensor.append(_ad)
    drug_FeatureTensor.append(_fe)

drug_AdjacencyTensor = np.array(drug_AdjacencyTensor)
drug_FeatureTensor = np.array(drug_FeatureTensor)

vae = load_model("./model/drug-molecule-generation-with-VAE",
               compile=False)

z_mean, _ = vae.encoder.predict([drug_AdjacencyTensor[:1000], drug_FeatureTensor[:1000]])

6/6 [==============================] - 0s 3ms/step


2023-03-02 22:05:14.437570: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [30]:
drug_feature_df = pd.DataFrame(data = z_mean, index=df['CID'])

In [31]:
drug_feature_df.head(5) # 435-dim vector

,0,1,2,3,4,5,6,7,8,9,...,425,426,427,428,429,430,431,432,433,434
CID,,,,,,,,,,,,,,,,,,,,,
10096043,0.007358,0.075078,-0.163587,0.025831,-0.040191,0.000627,0.162851,-0.044673,-0.423222,0.086359,...,0.006992,-0.151789,-0.248238,-0.054349,0.015753,-0.118295,0.103272,-0.210073,0.110451,-0.069613
46843057,0.007230,0.058161,-0.132759,0.043585,-0.051195,0.003567,0.144266,-0.038809,-0.369100,0.088933,...,-0.002204,-0.125048,-0.223964,-0.052225,0.012307,-0.093963,0.080036,-0.179362,0.091908,-0.062549
44224160,0.031977,0.087915,-0.148683,0.031577,-0.012937,-0.016674,0.138326,-0.023842,-0.369678,0.073843,...,0.005214,-0.124458,-0.226131,-0.047121,0.012374,-0.098900,0.091018,-0.172184,0.095501,-0.060776
24776445,0.005402,0.045830,-0.131909,0.058264,-0.074162,0.021043,0.151301,-0.050268,-0.393139,0.119950,...,-0.014712,-0.129678,-0.244254,-0.052003,0.012882,-0.095890,0.072875,-0.193940,0.102836,-0.066884
46883536,0.020644,0.069331,-0.132027,0.019614,-0.014420,-0.011757,0.123797,-0.026692,-0.325680,0.059359,...,0.012262,-0.107285,-0.194845,-0.042815,0.012381,-0.085350,0.085554,-0.156614,0.084245,-0.050680


## CNA 和 Gene-Expression dimension reduction

In [32]:
print(cna_df.shape)
cna_df.head()

(866, 24502)


,A1BG,A1CF,A2M,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
celline_barcode,,,,,,,,,,,,,,,,,,,,,
NCI-H1048,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
NCI-H1963,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
DoTc2-4510,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,3.0,...,4.0,3.0,3.0,5.0,5.0,3.0,5.0,2.0,2.0,3.0
GA-10,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
ZR-75-30,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0


In [33]:
print(fpkm_df.shape)
fpkm_df.head()

(866, 37602)


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,...,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,ZZZ3,RSKR,SHLD3,ATP6V1FNB
celline_barcode,,,,,,,,,,,,,,,,,,,,,
NCI-H1048,0.16,1.13,0.07,0.3,0.19,0.01,0.04,1.19,0.03,0.07,...,0.06,0.0,5.97,17.48,0.0,6.66,9.62,2.35,1.91,0.35
NCI-H1963,0.82,1.66,0.02,0.01,0.01,0.0,0.0,1.39,0.0,0.11,...,2.49,0.0,8.88,0.26,0.0,8.11,9.07,5.1,1.59,0.64
DoTc2-4510,0.33,1.02,0.0,0.0,0.08,1.08,0.03,0.03,0.01,0.21,...,6.39,0.0,12.85,15.08,0.0,2.35,6.33,1.74,1.22,0.11
GA-10,0.1,0.82,0.0,0.04,0.0,0.01,0.0,0.79,0.01,0.0,...,0.01,0.0,2.19,0.3,0.0,5.22,8.14,5.38,3.06,0.24
ZR-75-30,0.17,3.88,0.0,0.1,0.29,0.31,0.04,0.0,0.02,0.01,...,2.54,0.0,6.25,19.71,0.0,2.83,5.91,18.39,0.85,0.05


**CNA and FPKM data integration using SNF**

SNFtool: https://doi.org/10.1038/nmeth.2810

R code is in current folder/SNL_integration.R

In [34]:
!R CMD BATCH ./SNL_integration.R

In [35]:
similarity_df = pd.read_csv("./data/processed_data/simlilarity_matrix.csv")

In [36]:
similarity_df.drop(columns=['Unnamed: 0'], inplace=True)
similarity_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V857,V858,V859,V860,V861,V862,V863,V864,V865,V866
0,0.568480,0.002736,0.000742,0.001109,0.000992,0.000964,0.001446,0.000952,0.001119,0.001042,...,0.001185,0.000759,0.004580,0.001012,0.000799,0.000911,0.001059,0.000947,0.000993,0.000903
1,0.002736,0.579131,0.000918,0.000896,0.000799,0.000895,0.000869,0.000800,0.000823,0.000843,...,0.000885,0.000778,0.001108,0.000797,0.000858,0.000999,0.000893,0.001052,0.000878,0.001004
2,0.000742,0.000918,0.575670,0.000720,0.000953,0.001003,0.000758,0.001000,0.000937,0.000844,...,0.000651,0.001647,0.000821,0.000960,0.001169,0.001005,0.001079,0.000920,0.000949,0.000990
3,0.001109,0.000896,0.000720,0.551266,0.000804,0.000912,0.001281,0.000780,0.000828,0.001184,...,0.002038,0.000700,0.000975,0.000767,0.000847,0.000858,0.000788,0.000894,0.001016,0.000886
4,0.000992,0.000799,0.000953,0.000804,0.576983,0.000881,0.000853,0.003715,0.004447,0.001172,...,0.000753,0.000941,0.002537,0.004274,0.000938,0.000933,0.000976,0.000839,0.000879,0.000871


In [37]:
similarity_df.columns = cna_df.index

In [38]:
similarity_df.head() # 就是细胞的Genomic Profiles的特征向量
celline_feature = {}
for i, celline in enumerate(similarity_df.columns):
    celline_feature[celline] = np.array(similarity_df.iloc[i].values)

In [39]:
s = []
for i in response['sample_barcode']:
    celline_name, pubchem_id = i.split('_')
    celline_feature_array = celline_feature[celline_name]
    drug_feature_array = drug_feature_df.loc[int(pubchem_id)]
    combined_feature = np.hstack([celline_feature_array, drug_feature_array])
    s.append(combined_feature)
    

In [40]:
np.array(s).shape # 866+435=1301-dim vector

(145007, 1301)

In [41]:
np.save("./data/processed_data/all_feature", np.array(s))